# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [72]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import sklearn
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [73]:
sales = pd.read_csv('kc_house_data.csv')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [74]:
 train_data = pd.read_csv('kc_house_train_data.csv')
len(train_data)
test_data = pd.read_csv('kc_house_test_data.csv')
len(test_data)

4229

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [75]:

example_features = ['sqft_living', 'bedrooms', 'bathrooms']


lm = LinearRegression()
#model1:
X = train_data[example_features]  
Y = train_data['price']
lm.fit(X,Y)
print "estimated intercept " + str(lm.intercept_)
pd.DataFrame(zip(X.columns,lm.coef_),columns = ['feature','estimated coefficients'])


estimated intercept 87912.865815


,feature,estimated coefficients
0,sqft_living,315.406691
1,bedrooms,-65081.887116
2,bathrooms,6942.165986


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [76]:
 example_predictions = lm.predict(X)
print example_predictions[0] # should be 271789.505878


271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [77]:
def get_residual_sum_of_squares(model, data, outcome):
        # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - predictions
    # Then square and add them up
    RSS = (residuals * residuals).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [78]:

rss_example_train = get_residual_sum_of_squares(lm, test_data[example_features], test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [79]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [80]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [81]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + train_data['long']



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [82]:
model2_feature = ['bedrooms_squared','bed_bath_rooms','log_sqft_living','lat_plus_long']
model2Test = test_data[model2_feature]
averagevalue = model2Test.mean()
print averagevalue

bedrooms_squared    12.446678
bed_bath_rooms       7.503902
log_sqft_living      7.550275
lat_plus_long      -74.653175
dtype: float64


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [83]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [101]:
# Learn the three models: (don't forget to set validation_set = None)
#model1
X_model1 = train_data[model_1_features]
lm.fit(X_model1,Y)
model1_coef = lm.coef_
model1_inter = lm.intercept_
X_test1 = test_data[model_1_features]

RSS1 = get_residual_sum_of_squares(lm, X_test1, test_data['price'])
print RSS1, lm.coef_, lm.intercept_
#model2
X_model2 = train_data[model_2_features]
lm.fit(X_model2,Y)
model2_coef = lm.coef_
model2_inter = lm.intercept_
X_test2 = test_data[model_2_features]

RSS2 = get_residual_sum_of_squares(lm, X_test2, test_data['price'])
print RSS2, lm.coef_, lm.intercept_

# model 3
X_model3 = train_data[model_3_features]
lm.fit(X_model3,Y)
model3_coef = lm.coef_
model3_inter = lm.intercept_
X_test3 = test_data[model_3_features]

RSS3 = get_residual_sum_of_squares(lm, X_test3, test_data['price'])
print RSS3, lm.coef_, lm.intercept_


2.25500469795e+14 [  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05] -69075726.7926
2.23377462976e+14 [  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04] -66867968.8711
2.61942651061e+14 [  5.29422820e+02   3.45142296e+04   6.70607813e+04   5.34085611e+05
  -4.06750711e+05  -8.57050439e+03  -6.78858667e+03  -5.61831484e+05
   1.27334900e+05] -62036084.9861


In [98]:
# Examine/extract each model's coefficients:
Coef = pd.DataFrame(zip(X_model3.columns,model1_coef,model2_coef,model3_coef),columns = ['feature','model1','model2','model3'])
intercept_row = ['intercept',model1_inter,model2_inter,model3_inter]
Coef.loc['5'] = intercept_row
RSS_row = ['RSS',RSS1,RSS2,RSS3]
Coef.loc['6'] = RSS_row

print Coef

       feature        model1        model2        model3
0  sqft_living  3.122586e+02  3.066101e+02  5.294228e+02
1     bedrooms -5.958653e+04 -1.134464e+05  3.451423e+04
2    bathrooms  1.570674e+04 -7.146131e+04  6.706078e+04
3          lat  6.586193e+05  6.548446e+05  5.340856e+05
4         long -3.093744e+05 -2.942990e+05 -4.067507e+05
4    intercept -6.907573e+07 -6.686797e+07 -6.203608e+07
5          RSS  2.255005e+14  2.233775e+14  2.619427e+14


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [102]:
print RSS1
print RSS2
print RSS3

2.25500469795e+14
2.23377462976e+14
2.61942651061e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [103]:
test_data['bedrooms_squared'].mean()

12.4466777015843

In [104]:
test_data['bed_bath_rooms'].mean()


7.5039016315913925

In [105]:
test_data['log_sqft_living'].mean()


7.550274679645921